In [ ]:
from tensorflow import keras
import numpy as np


**Q1**

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
validation_images=train_images[:6000]
validation_labels=train_labels[:6000]
train_images=train_images[6000:]
train_labels=train_labels[6000:]

4422102/4422102 [==============================] - 0s 0us/step


**Q2**

In [ ]:
class ActivationFn:
  def sigmoid(self,layer,d=False):
      if(d==True):
          return self.sigmoid(layer)*(1-self.sigmoid(layer))
      return 1.0/(1.0+np.exp(-1.0*layer))

  def softmax(self,layer):
      #print(layer)
      newlayer=(layer-np.max(layer))
      #print(new_layer)
      return np.exp(newlayer)/np.sum(np.exp(newlayer))

  # def d_sigmoid(layer):
  #     return sigmoid(layer)*(1-sigmoid(layer))

  def cross_entropy(self,true_output,output):
      return -1.0*np.sum(true_output * np.log(output+1e-9))

  def tanh(self,layer,d=False):
      if(d==True):
          return 1-(np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))**2
      return (np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))

  # def d_tanh(layer):
  #     return 1-(np.exp(layer)-np.exp(-layer))/(np.exp(layer)+np.exp(-layer))**2

  def relu(self,layer,d=False):
      if(d==True):
        return 1. * (layer > 0)
      return layer * (layer > 0)

  # def d_relu(layer):
  #     return 1. * (layer > 0)

In [ ]:
input_size=28*28
output_size=10
num_hidden_layers=2
num_neurons_in_hidden_layer=16
pre_activation_layer={}
activation_layer={}
neurons_list=[]
weights={}
bias={}
input_layer=np.random.rand(28*28,1)#to be defined n*1
# output_layer=np.zeros((output_labels,1))

neurons_list.append(input_size)
for i in range(num_hidden_layers):
  neurons_list.append(num_neurons_in_hidden_layer)
neurons_list.append(output_size)

#hidden_layer[0]=input_layer
# for i in range(num_hidden_layers):
#   hidden_layer.append(np.zeros(1,num_neurons_in_hidden_layer))
# hidden_layer.append(output_layer)

activation_layer[0]=input_layer
# for i in range(num_hidden_layers):
#   activation_layer.append(np.zeros(1,num_neurons_in_hidden_layer))
# activation_layer.append(output_layer)

for i in range(len(neurons_list)-1):
  weights[i+1]=np.random.rand(neurons_list[i+1],neurons_list[i])

for i in range(len(neurons_list)-1):
  bias[i+1]=np.random.rand(neurons_list[i+1],1)


def forward_propogation():
  for i in range(1,num_hidden_layers+1):
    pre_activation_layer[i]=bias[i]+np.matmul(weights[i],activation_layer[i-1])
    activation_layer[i]=sigmoid(pre_activation_layer[i])
  pre_activation_layer[num_hidden_layers+1]=bias[num_hidden_layers+1]+np.matmul(weights[num_hidden_layers+1],activation_layer[num_hidden_layers])
  activation_layer[num_hidden_layers+1]=softmax(pre_activation_layer[num_hidden_layers+1])

forward_propogation()
  

In [ ]:
class FeedFowardNeuralNetwork:
  def __init__(self,num_hidden_layers,num_neurons_in_hidden_layer):
    self.num_hidden_layers=num_hidden_layers
    self.num_neurons_in_hidden_layer=num_neurons_in_hidden_layer
    
  def weight_initialization(self):
    self.pre_activation_layer={}
    self.activation_layer={}
    self.weights={}
    self.bias={}
    self.neurons_list=[self.input_size]

    for i in range(self.num_hidden_layers):
      self.neurons_list.append(self.num_neurons_in_hidden_layer)
    self.neurons_list.append(self.output_size)

    self.activation_layer[0]=self.input_layer

    for i in range(len(self.neurons_list)-1):
      self.weights[i+1]=np.random.rand(self.neurons_list[i+1],self.neurons_list[i])

    for i in range(len(self.neurons_list)-1):
      self.bias[i+1]=np.random.rand(self.neurons_list[i+1],1)
  
  def forward_propogation(self):
    for i in range(1,self.num_hidden_layers+1):
      self.pre_activation_layer[i]=self.bias[i]+np.matmul(self.weights[i],self.activation_layer[i-1])
      self.activation_layer[i]=sigmoid(self.pre_activation_layer[i])
    self.pre_activation_layer[self.num_hidden_layers+1]=self.bias[self.num_hidden_layers+1]+np.matmul(self.weights[self.num_hidden_layers+1],self.activation_layer[self.num_hidden_layers])
    self.activation_layer[self.num_hidden_layers+1]=softmax(self.pre_activation_layer[self.num_hidden_layers+1])    
    return self.activation_layer[self.num_hidden_layers+1]

  def run(self,X,Y):
    self.input_layer=X
    self.input_size=len(X)
    self.output_size=Y
    self.weight_initialization()
    return self.forward_propogation()


nn=FeedFowardNeuralNetwork(1,128)
print(nn.run(train_images[0].reshape((-1,1)),10))# Converting to column vector



In [ ]:
class BackpropogationNN:
  def __init__(self,num_hidden_layers,num_neurons_in_hidden_layer):
    self.num_hidden_layers=num_hidden_layers
    self.num_neurons_in_hidden_layer=num_neurons_in_hidden_layer
    self.learning_rate=0.1
    self.epoch=15
    self.batch=32
    # self.activation_f=act
    self.pre_activation_layer={}
    self.activation_layer={}
    self.weights={}
    self.bias={}
    self.d_pre_activation_layer={}
    self.d_activation_layer={}
    self.d_weights={}
    self.d_bias={}

  def weight_initialization(self):
    self.neurons_list=[self.input_size]

    for i in range(self.num_hidden_layers):
      self.neurons_list.append(self.num_neurons_in_hidden_layer)
    self.neurons_list.append(self.output_size)

    #self.pre_activation_layer[0]=self.input_layer #to be checked

    for i in range(len(self.neurons_list)-1):
      self.weights[i+1]=np.random.uniform(-1,1,(self.neurons_list[i+1],self.neurons_list[i]))

    for i in range(len(self.neurons_list)-1):
      self.bias[i+1]=np.random.uniform(-1,1,(self.neurons_list[i+1],1))
  
  def forward_propogation(self,x):
    self.activation_layer[0]=x
    for i in range(1,self.num_hidden_layers+1):
      self.pre_activation_layer[i]=self.bias[i]+np.matmul(self.weights[i],self.activation_layer[i-1])
      self.activation_layer[i]=sigmoid(self.pre_activation_layer[i])
    self.pre_activation_layer[self.num_hidden_layers+1]=self.bias[self.num_hidden_layers+1]+np.matmul(self.weights[self.num_hidden_layers+1],self.activation_layer[self.num_hidden_layers])
    self.activation_layer[self.num_hidden_layers+1]=softmax(self.pre_activation_layer[self.num_hidden_layers+1])    
    return self.activation_layer[self.num_hidden_layers+1]

  def back_propogation(self,true_output,output):
    d_pre_activation_layer={}
    d_weights={}
    d_activation_layer={}
    d_bias={}
    d_pre_activation_layer[self.num_hidden_layers+1]=-1*(true_output-output)
    for i in range(self.num_hidden_layers+1,0,-1):
      d_weights[i]=np.outer(d_pre_activation_layer[i],self.activation_layer[i-1].transpose())
      d_bias[i]=d_pre_activation_layer[i]
      if(i==1):break
      d_activation_layer[i-1]=np.matmul(self.weights[i].transpose(),d_pre_activation_layer[i])
      d_pre_activation_layer[i-1]=np.multiply(d_activation_layer[i-1],d_sigmoid(self.pre_activation_layer[i-1]))#element multiplication
      #condn for index 0
    return d_weights,d_bias,cross_entropy(true_output,output)
    
  def update_parameters(self):
    self.d_weights = {k: v / self.batch for k, v in self.d_weights.items()}
    self.d_bias = {k: v / self.batch for k, v in self.d_bias.items()}
    for i in range(1,self.num_hidden_layers+2):
      self.weights[i]=self.weights[i]-self.learning_rate*self.d_weights[i]
    for i in range(1,self.num_hidden_layers+2):
      self.bias[i]=self.bias[i]-self.learning_rate*self.d_bias[i]

  def flush_gradients(self):
    self.d_weights={}
    self.d_bias={}

  def run(self,train_images,train_labels):
    self.input_size=train_images.shape[1]*train_images.shape[2]
    self.output_size=10
    self.weight_initialization()
    for epoch_no in range(self.epoch):
      loss=0
      batch_count=0
      for i in range(train_images.shape[0]):
        batch_count+=1
        #self.input_layer=train_images[i].reshape((-1,1))
        true_output=np.zeros((self.output_size,1))
        true_output[train_labels[i]-1]=1

        output=self.forward_propogation(train_images[i].reshape((-1,1)))
        #print(np.argmax(output))
        d_weights,d_bias,cur_loss=self.back_propogation(true_output,output)
        # if(epoch_no==1):
        #    print(d_weights)
        loss+=cur_loss
        for j in range(1,self.num_hidden_layers+2):
          if self.d_weights.get(j) is not None:
            self.d_weights[j]=self.d_weights[j]+d_weights[j]
          else:
            self.d_weights[j]=d_weights[j]
          if self.d_bias.get(j) is not None:
            self.d_bias[j]=self.d_bias[j]+d_bias[j]
          else:
            self.d_bias[j]=d_bias[j]
        
        if(batch_count==self.batch):
          batch_count=0
          self.update_parameters()
          self.flush_gradients()

      if(batch_count>0):#remaining
          batch_count=0
          self.update_parameters()
          self.flush_gradients()        

    
      print(epoch_no+1,loss/train_images.shape[0])
      #self.d_weights = {k: v / train_images.shape[0] for k, v in self.d_weights.items()}
      #self.d_bias = {k: v / train_images.shape[0] for k, v in self.d_bias.items()}
      # self.update_parameters()
      self.flush_gradients()




nn=BackpropogationNN(3,128)
nn.run(train_images,train_labels)# Converting to column vector



In [ ]:
class BackpropogationNN:
  actobj=ActivationFn()
  def __init__(self,num_hidden_layers,num_neurons_in_hidden_layer,learning_rate,epoch,batch,activation,beta,optimizer,init_strat):
    self.num_hidden_layers=num_hidden_layers
    self.num_neurons_in_hidden_layer=num_neurons_in_hidden_layer
    self.learning_rate=learning_rate
    self.epoch=epoch
    self.batch=batch
    self.pre_activation_layer={}
    self.activation_layer={}
    self.weights={}
    self.bias={}
    self.d_pre_activation_layer={}
    self.d_activation_layer={}
    self.d_weights={}
    self.d_bias={}
    self.beta=beta
    self.beta1=0.999
    self.optimizer=optimizer
    self.init_strat=init_strat
    # self.activation_fn=activation
    if(activation=="sigmoid"):self.activation_fn=BackpropogationNN.actobj.sigmoid
    if(activation=="tanh"):self.activation_fn=BackpropogationNN.actobj.tanh
    if(activation=="relu"):self.activation_fn=BackpropogationNN.actobj.relu

  def weight_initialization(self):
    self.neurons_list=[self.input_size]

    for i in range(self.num_hidden_layers):
      self.neurons_list.append(self.num_neurons_in_hidden_layer)
    self.neurons_list.append(self.output_size)

    for i in range(len(self.neurons_list)-1):
      if(self.init_strat=="random"):
        self.weights[i+1]=np.random.default_rng().uniform(-1,1,(self.neurons_list[i+1],self.neurons_list[i]))
        self.bias[i+1]=np.random.default_rng().uniform(-1,1,(self.neurons_list[i+1],1))
      elif(self.init_strat=="xavier"):
        self.weights[i+1]=np.random.randn(self.neurons_list[i+1],self.neurons_list[i])*np.sqrt(2/(self.neurons_list[i+1]+self.neurons_list[i]))
        self.bias[i+1]=np.random.randn(self.neurons_list[i+1],1)*np.sqrt(2/(self.neurons_list[i+1]+1))
  
  def forward_propogation(self,x):
    self.activation_layer[0]=x
    for i in range(1,self.num_hidden_layers+1):
      self.pre_activation_layer[i]=self.bias[i]+np.matmul(self.weights[i],self.activation_layer[i-1])
      self.activation_layer[i]=self.activation_fn(self.pre_activation_layer[i])
    self.pre_activation_layer[self.num_hidden_layers+1]=self.bias[self.num_hidden_layers+1]+np.matmul(self.weights[self.num_hidden_layers+1],self.activation_layer[self.num_hidden_layers])
    self.activation_layer[self.num_hidden_layers+1]=BackpropogationNN.actobj.softmax(self.pre_activation_layer[self.num_hidden_layers+1])    
    return self.activation_layer[self.num_hidden_layers+1]

  def back_propogation(self,true_output,output):
    d_pre_activation_layer={}
    d_weights={}
    d_activation_layer={}
    d_bias={}
    d_pre_activation_layer[self.num_hidden_layers+1]=-1*(true_output-output)
    for i in range(self.num_hidden_layers+1,0,-1):
      d_weights[i]=np.outer(d_pre_activation_layer[i],self.activation_layer[i-1].T)
      d_bias[i]=d_pre_activation_layer[i]
      if(i==1):break
      d_activation_layer[i-1]=np.matmul(self.weights[i].T,d_pre_activation_layer[i])
      d_pre_activation_layer[i-1]=np.multiply(d_activation_layer[i-1],self.activation_fn(self.pre_activation_layer[i-1],True))#element multiplication

    return d_weights,d_bias,BackpropogationNN.actobj.cross_entropy(true_output,output)
    

  def flush_gradients(self):
    for i in range(len(self.neurons_list)-1):
      self.d_weights[i+1]=np.zeros((self.neurons_list[i+1],self.neurons_list[i]))
      self.d_bias[i+1]=np.zeros((self.neurons_list[i+1],1))

  def accuracy(self,images,labels):
    count=0
    loss=0
    for i in range(images.shape[0]):
      output=self.forward_propogation(images[i].reshape((-1,1)))
      true_output=np.zeros((self.output_size,1))
      true_output[labels[i]]=1
      if(labels[i]==np.argmax(output)):
        count+=1
      loss+=BackpropogationNN.actobj.cross_entropy(true_output,output)

    return count*1.0/images.shape[0], loss/images.shape[0]
    

  def run(self,train_images,train_labels,validation_images,validation_labels):
    self.input_size=train_images.shape[1]*train_images.shape[2]
    self.output_size=10
    self.weight_initialization()

    if self.optimizer=="sgd":
      opt=sgd()
    elif self.optimizer=="momentum":
      opt=momentum(self.neurons_list)
    elif self.optimizer=="rmsprop":
      opt=rmsprop(self.neurons_list)
    elif self.optimizer=="nag":
      opt=nag(self.neurons_list,self.weights,self.bias)
    elif self.optimizer=="adam":
      opt=adam(self.neurons_list)
    elif self.optimizer=="nadam":
      opt=nadam(self.neurons_list)

    t=0
    for epoch_no in range(self.epoch):
      val_loss,tra_loss,tra_acc,val_acc=0,0,0,0
      loss=0
      batch_count=0
      self.flush_gradients()
      for i in range(train_images.shape[0]):
        batch_count+=1
        true_output=np.zeros((self.output_size,1))
        true_output[train_labels[i]]=1
        output=self.forward_propogation(train_images[i].reshape((-1,1)))
        d_weights,d_bias,cur_loss=self.back_propogation(true_output,output)
        loss+=cur_loss
        for j in range(1,self.num_hidden_layers+2):
            self.d_weights[j]=self.d_weights[j]+d_weights[j]
            self.d_bias[j]=self.d_bias[j]+d_bias[j]


        if(batch_count==self.batch):
          self.d_weights = {k: v / self.batch for k, v in self.d_weights.items()}
          self.d_bias = {k: v / self.batch for k, v in self.d_bias.items()}
          batch_count=0
          if self.optimizer=="sgd":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias)
          elif self.optimizer=="momentum":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="rmsprop":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="adam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nadam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nag":
            self.weights,self.bias,self.weights_use,self.bias_use=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)

          self.flush_gradients()

      if(batch_count>0):#remaining
          self.d_weights = {k: v / batch_count for k, v in self.d_weights.items()}
          self.d_bias = {k: v / batch_count for k, v in self.d_bias.items()}
          batch_count=0
          if self.optimizer=="sgd":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias)
          elif self.optimizer=="momentum":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="rmsprop":
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)
          elif self.optimizer=="adam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nadam":
            t+=1
            self.weights,self.bias=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta,self.beta1,t)
          elif self.optimizer=="nag":
            self.weights,self.bias,self.weights_use,self.bias_use=opt.optimize(self.num_hidden_layers,self.weights,self.bias,self.learning_rate,self.d_weights,self.d_bias,self.beta)

          self.flush_gradients()     

      val_acc,val_loss=self.accuracy(validation_images,validation_labels)
      tra_acc,tra_loss=self.accuracy(train_images,train_labels) 
      print(epoch_no+1,'Training loss',tra_loss,'Val loss',val_loss,'Training Accuracy',tra_acc,'Val Accuracy',val_acc)
      #wandb.log({"Training loss":tra_loss,'Val loss':val_loss,'Training Accuracy':tra_acc,'Val Accuracy':val_acc})
    if self.optimizer=="nag":
      self.weights=self.weights_use #for nag
      self.bias=self.bias_use #for nag


#num_hidden_layers,num_neurons_in_hidden_layer,learning_rate,epoch,batch,activation,beta,optimizer
nn=BackpropogationNN(3,128,0.0001,10,32,"relu",0.9,"adam","xavier")
nn.run(train_images,train_labels,validation_images,validation_labels)



In [ ]:
class sgd:
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias):
    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*d_weights[i]
      bias[i]=bias[i]-learning_rate*d_bias[i]
    return weights,bias

class momentum:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+d_weights[i]
        self.update_b[i]=beta*self.update_b[i]+d_bias[i]

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*self.update_w[i]
      bias[i]=bias[i]-learning_rate*self.update_b[i]

    return weights,bias

class nag:
  def __init__(self,neurons_list,weights,bias):
    self.update_w={}
    self.update_b={}
    self.weights_use={}
    self.bias_use={}
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
    self.bias_use=bias.copy()
    self.weights_use=weights.copy()
  
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+d_weights[i]
        self.update_b[i]=beta*self.update_b[i]+d_bias[i]

    for i in range(1,num_hidden_layers+2):
      self.weights_use[i]=self.weights_use[i]-learning_rate*self.update_w[i]
      self.bias_use[i]=self.bias_use[i]-learning_rate*self.update_b[i]

    for i in range(1,num_hidden_layers+2):
      weights[i]=self.weights_use[i]-beta*self.update_w[i]
      bias[i]=self.bias_use[i]-beta*self.update_b[i] 

    return weights,bias,self.weights_use,self.bias_use

class rmsprop:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))

  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta):
    for i in range(1,num_hidden_layers+2):
        self.update_w[i]=beta*self.update_w[i]+(1-beta)*((d_weights[i])**2)
        self.update_b[i]=beta*self.update_b[i]+(1-beta)*((d_bias[i])**2)

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*d_weights[i]/(np.sqrt(self.update_w[i])+self.epsilon)
      bias[i]=bias[i]-learning_rate*d_bias[i]/(np.sqrt(self.update_b[i])+self.epsilon)
      
    return weights,bias



class adam:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.update_what={}
    self.update_bhat={}
    self.momentum_w={}
    self.momentum_b={}
    self.momentum_what={}
    self.momentum_bhat={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
      self.momentum_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.momentum_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta1,beta2,t):
    for i in range(1,num_hidden_layers+2):
        self.momentum_w[i]=beta1*self.momentum_w[i]+(1-beta1)*((d_weights[i]))
        self.momentum_what[i]=self.momentum_w[i]/(1-beta1**t)

        self.momentum_b[i]=beta1*self.momentum_b[i]+(1-beta1)*((d_bias[i]))
        self.momentum_bhat[i]=self.momentum_b[i]/(1-beta1**t)

        self.update_w[i]=beta2*self.update_w[i]+(1-beta2)*((d_weights[i])**2)
        self.update_what[i]=self.update_w[i]/(1-beta2**t)

        self.update_b[i]=beta2*self.update_b[i]+(1-beta2)*((d_bias[i])**2)
        self.update_bhat[i]=self.update_b[i]/(1-beta2**t)
  

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate*self.momentum_what[i]/(np.sqrt(self.update_what[i])+self.epsilon)
      bias[i]=bias[i]-learning_rate*self.momentum_bhat[i]/(np.sqrt(self.update_bhat[i])+self.epsilon)
      
    return weights,bias
    
    

class nadam:
  def __init__(self,neurons_list):
    self.update_w={}
    self.update_b={}
    self.update_what={}
    self.update_bhat={}
    self.momentum_w={}
    self.momentum_b={}
    self.momentum_what={}
    self.momentum_bhat={}
    self.epsilon=1e-6
    for i in range(len(neurons_list)-1):
      self.update_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.update_b[i+1]=np.zeros((neurons_list[i+1],1))
      self.momentum_w[i+1]=np.zeros((neurons_list[i+1],neurons_list[i]))
      self.momentum_b[i+1]=np.zeros((neurons_list[i+1],1))
  def optimize(self,num_hidden_layers,weights,bias,learning_rate,d_weights,d_bias,beta1,beta2,t):
    for i in range(1,num_hidden_layers+2):
        self.momentum_w[i]=beta1*self.momentum_w[i]+(1-beta1)*((d_weights[i]))
        self.momentum_what[i]=self.momentum_w[i]/(1-beta1**t)

        self.momentum_b[i]=beta1*self.momentum_b[i]+(1-beta1)*((d_bias[i]))
        self.momentum_bhat[i]=self.momentum_b[i]/(1-beta1**t)

        self.update_w[i]=beta2*self.update_w[i]+(1-beta2)*((d_weights[i])**2)
        self.update_what[i]=self.update_w[i]/(1-beta2**t)

        self.update_b[i]=beta2*self.update_b[i]+(1-beta2)*((d_bias[i])**2)
        self.update_bhat[i]=self.update_b[i]/(1-beta2**t)
  

    for i in range(1,num_hidden_layers+2):
      weights[i]=weights[i]-learning_rate/(np.sqrt(self.update_what[i])+self.epsilon)*(beta1*self.momentum_what[i]+(1-beta1)*d_weights[i]/(1-beta1**t))
      bias[i]=bias[i]-learning_rate/(np.sqrt(self.update_bhat[i])+self.epsilon)*(beta1*self.momentum_bhat[i]+(1-beta1)*d_bias[i]/(1-beta1**t))
      
    return weights,bias
  

